https://youtu.be/DxygPxcfW_I

# **This code is an implementation of a chatbot using the GPT-2 language model from Hugging Face's Transformers library.**
<br>
Reference: https://huggingface.co/transformers/v2.2.0/pretrained_models.html



In [ ]:
# @title Load Configuraiton 
import hbqaconfig as conf

In [ ]:
# !pip install transformers
# !pip install torch
# !pip install -U PyPDF2
# !pip install python-docx
# !pip install huggingface
# !pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 29.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184487 sha256=5b05057e61deab0b59e4a0c9ca1183e71308b067b826aa88f02355e35beb8cdb
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
# !pip install accelerate>=0.20.1
# !pip install transformers[torch]
# # You need to restart the kernel after this step

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
apipath = r'H:\\My Drive\\config\\hbqa.txt'
apipath = r'/content/drive/MyDrive/config/hbqa-colab.txt'
import configparser

config = configparser.ConfigParser()
config.read(apipath)

secret_key = config['global']['OPENAI_KEY']
datapath = config['global']['DATA_FOLDER']
corpuspath = config['global']['CORPUS_FOLDER']
corpus_sectionpath = config['global']['CORPUS_FOLDER_SECTIONS']
PINECONE_API_KEY = config['global']['PINECONE_API_KEY']
# Original corpus in the form of pdf or text or doc file. Originally this is from github.
PINECONE_ENV = config['global']['PINECONE_ENV']
CHATGPT_KEY = config['global']['CHATGPT_KEY']
WANDB_KEY= config['global']['WANDB_KEY']

# PINECONE is Vector Database. To store the vector so that we can quickly search the vector space.
# https://app.pinecone.io
# get PINECONE_API_KEY key from app.pinecone.io
# find your PINECONE_ENVIRONMENT next to the api key in pinecone console

corpuspath = datapath + "MBBook-from-HTML-Chapters/Trainfiles"
modelpath  = datapath + "MBBook-from-HTML-Chapters/Trainfiles/gpt2"

In [ ]:
import os
import re
from PyPDF2 import PdfReader
import docx
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb


In [ ]:
import torch

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")
print(DEVICE)

cpu


In [ ]:
wandb.login(key=WANDB_KEY)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# !huggingface-cli login

Required functions to read text from various files located in a directory. Files can be a mix of pdf, docx, or txt.

In [ ]:
# Functions to read different file types
def read_pdf(file_path):
    with open(file_path, "rb") as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

def read_word(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith(".pdf"):
            combined_text += read_pdf(file_path)
        elif filename.endswith(".docx"):
            combined_text += read_word(file_path)
        elif filename.endswith(".txt"):
            combined_text += read_txt(file_path)
    return combined_text



The train_chatbot function uses the combined text data to train a GPT-2 model using the provided training arguments. The resulting trained model and tokenizer are then saved to a specified output directory.

In [ ]:
wandb.init(project="DBA", name="HBQA_GPT2_Medium")

wandb: Currently logged in as: hari-thapliyal. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.10
wandb: Run data is saved locally in /content/wandb/run-20230921_063510-ffeh6r55
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run HBQA_GPT2_Medium
wandb: ⭐️ View project at https://wandb.ai/hari-thapliyal/DBA
wandb: 🚀 View run at https://wandb.ai/hari-thapliyal/DBA/runs/ffeh6r55


## Training

In [ ]:
from transformers import EarlyStoppingCallback

In [ ]:
# Create an instance of the EarlyStoppingCallback
Early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Adjust the patience value as needed
    early_stopping_threshold=None,  # You can set a threshold for improvement if desired
)

callback_list = [Early_stopping_callback]

In [ ]:
#medium model error
# CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 14.75 GiB total capacity;
# 12.79 GiB already allocated; 832.00 KiB free; 13.72 GiB reserved in total by PyTorch)
# If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.
# See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

#large model error
# CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 14.75 GiB total capacity;
#12.78 GiB already allocated; 832.00 KiB free; 13.72 GiB reserved in total by PyTorch)
# If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.
# See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
%time

# Read documents from the directory
combined_text = read_documents_from_directory(corpuspath)
combined_text = re.sub(r'\n+', '\n', combined_text).strip()  # Remove excess newline characters

# Split the text into training and validation sets
train_fraction=0.8
split_index = int(train_fraction * len(combined_text))
train_text = combined_text[:split_index]
val_text = combined_text[split_index:]

# Save the training and validation data as text files
with open("train.txt", "w") as f:
    f.write(train_text)
with open("val.txt", "w") as f:
    f.write(val_text)

# Set up the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl
model = GPT2LMHeadModel.from_pretrained("gpt2-medium").to(DEVICE)  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl

# Prepare the dataset
train_dataset = TextDataset(tokenizer=tokenizer, file_path="train.txt", block_size=128)
val_dataset = TextDataset(tokenizer=tokenizer, file_path="val.txt", block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.3 µs


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# %time

# # Set up the training arguments
training_args = TrainingArguments(
    output_dir=modelpath,
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=25,
    save_steps=10000, # Set the frequency to save the model
    eval_steps=10000,  # Set the frequency to evaluate the model
    save_total_limit=1, # Limit the number of saved checkpoints to 1 (best model)
    logging_dir='./logs', # learning_rate=2e-5,
    evaluation_strategy="epoch",  # Change this to "steps"/ "epoch" if you want to evaluate at the end of each epoch
    remove_unused_columns=False,
    push_to_hub=False,  # Set to True if you want to push the model to the Hugging Face Hub
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,

)

trainer.train()
trainer.save_model(modelpath)

# Save the tokenizer
tokenizer.save_pretrained(modelpath)

Epoch,Training Loss,Validation Loss
1,3.335000,3.104804
2,2.796400,3.033715
3,2.467600,3.071103
4,2.206100,3.186008
5,1.976700,3.333058
6,1.525100,3.546866
7,1.335200,3.731298
8,1.186600,3.997883
9,1.054000,4.244314
10,0.931300,4.451337


('/content/drive/Othercomputers/Spectre2/DBA/Data/MBBook-from-HTML-Chapters/Trainfiles/gpt2/tokenizer_config.json',
 '/content/drive/Othercomputers/Spectre2/DBA/Data/MBBook-from-HTML-Chapters/Trainfiles/gpt2/special_tokens_map.json',
 '/content/drive/Othercomputers/Spectre2/DBA/Data/MBBook-from-HTML-Chapters/Trainfiles/gpt2/vocab.json',
 '/content/drive/Othercomputers/Spectre2/DBA/Data/MBBook-from-HTML-Chapters/Trainfiles/gpt2/merges.txt',
 '/content/drive/Othercomputers/Spectre2/DBA/Data/MBBook-from-HTML-Chapters/Trainfiles/gpt2/added_tokens.json')

## Earlier training
CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.39 µs
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
 [11198/59500 1:38:42 < 7:05:50, 1.89 it/s, Epoch 18.82/100]
Step	Training Loss
500	3.334900
1000	2.793100
1500	2.456100
2000	2.196700
2500	1.940500
3000	1.720800
3500	1.466000
4000	1.260700
4500	1.096900
5000	0.949300
5500	0.818100
6000	0.700800
6500	0.584900
7000	0.490200
7500	0.422300
8000	0.365900
8500	0.320800
9000	0.281000
9500	0.244800
10000	0.207200
10500	0.187400
11000	0.171300


# Testing Model Response

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel


In [ ]:
# The generate_response function takes a trained model, tokenizer, and a prompt string as input and generates a response using the GPT-2 model.

def generate_response(model, tokenizer, prompt, max_length=250):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained(modelpath).to(DEVICE)
tokenizer = GPT2Tokenizer.from_pretrained(modelpath)

In the case of the GPT-2 tokenizer, the model uses a byte-pair encoding (BPE) algorithm, which tokenizes text into subword units. As a result, one word might be represented by multiple tokens.

For example, if you set max_length to 50, the generated response will be limited to 50 tokens, which could be fewer than 50 words, depending on the text.

In [ ]:
prompt = "Who did the Muni ask for a bride and why?"  # Replace with your desired prompt
response = generate_response(model, tokenizer, prompt, max_length=100)  # This is small lenght question
print("Generated response:", response)

Generated response: Who did the Muni ask for a bride and why? Who was the Rishi whom he  desired to select? Why 
was he so highly regarded by the king? Why also was he so hated by the  wicked Rishi? I wish to hear all 
these in detail.' 
 "Sauti said, 'O thou whose wealth is asceticism, the subjects of this 
history are diverse. This is but the beginning. I shall narrate the whole


In [ ]:
prompt = "What did Mamata tell Vrihaspati regarding her condition?"  # Replace with your desired prompt
response = generate_response(model, tokenizer, prompt, max_length=200)  # This is small lenght question
print("Generated response:", response)

Generated response: What did Mamata tell Vrihaspati regarding her condition? 
 "Vrihaspati said, 'O best of Brahmanas, the Asura (mamata) who brought forth thy child 
158 through the connection of a Brahmana's daughter, became, O best of 
Brahmanas, thy husband. She gave birth to thy child. The child became 
exceedingly devoted to thee and to his mother. Indeed, becau se he was so devoted to 
you, he became your son in righteousness. O thou o f fair smiles, he was devoted to 
virtue, honesty, and truth. Even in his boyhoo d, he was impetuous and vain. O 
beautiful one, he was virtuous and truthful. In splend our he was like unto a golden 
mountain. In splendour he was like a blazing fire;  in loveliness and affluence


In [ ]:
prompt = "Can you ask one question from page 20 of this book?"  # Replace with your desired prompt
response = generate_response(model, tokenizer, prompt, max_length=200)  # This is small lenght question
print("Generated response:", response)

Generated response: Can you ask one question from page 20 of this book? 
 "Asked by a disciple of Vyasa, the illustrious Rishi  replied, 'O Bhi shma, the sun hath no 
power to set in the usual time, if the moon is down. O thou of ascetic wealth, 
the answer is simple: the four orders (celestials,  Asuras, Rakshasas, 
and men) depend on the lightness of mind which ser veth for success. If the 
person seeking to do good to his relatives by making a gift, or by 
sacrifices, or by attending to the poor, should the  sun go down, he can never 
succeed in accomplishing his object. The wise say that a person in 
woe should never yield to influence of evil,  or to the injunctions of his ancestors. The 
wise also say that a person
